### Install the required tools.

In [1]:
pip install pyModeS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 11.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


### Decode the given files.

In [6]:
import pyModeS as pms

# Input file path
log_file_path = "adsb_raw_data.txt"

# Output file paths
decoded_output_file = "decoded_messages.txt"
invalid_output_file = "invalid_messages.txt"

# Function to decode a single ADS-B message
def decode_adsb_message(message):
    try:
        if len(message) != 28:
            return {"Message": message, "Error": "Invalid length"}
        
        # Check CRC validity
        crc_check = pms.crc(message)
        if crc_check != 0:
            return {"Message": message, "Error": "CRC check failed"}

        icao = pms.icao(message)
        df = pms.df(message)
        msg_type = pms.typecode(message)

        decoded_info = {
            "Message": message,
            "ICAO Address": icao,
            "Downlink Format": df,
            "Type Code": msg_type,
        }

        # Decode additional information based on Type Code
        if msg_type in range(1, 5):  # Aircraft identification and category
            decoded_info["Callsign"] = pms.adsb.callsign(message)
        elif msg_type in range(9, 19):  # Airborne position
            decoded_info["Altitude"] = pms.adsb.altitude(message)
        elif msg_type == 19:  # Airborne velocity
            decoded_info["Velocity"] = pms.adsb.velocity(message)
        else:
            # Handle other message types with descriptions or context
            if msg_type in range(5, 9):  # Messages related to aircraft status, intent, etc.
                decoded_info["Details"] = f"Aircraft status, intent, or other non-position messages (Type {msg_type})"
            elif msg_type == 20:  # For example, surveillance information
                decoded_info["Details"] = "Surveillance information message"
            else:
                decoded_info["Details"] = f"Unrecognized message type (Type {msg_type})"

        return decoded_info
    except Exception as e:
        return {"Message": message, "Error": str(e)}

# Process the log file
decoded_messages = []
invalid_messages = []

try:
    with open(log_file_path, "r") as file:
        for line in file:
            raw_message = line.strip()  # Original line
            cleaned_message = raw_message.strip('*').strip(';')  # Cleaned message

            if len(cleaned_message) == 28:  # Validate message length
                result = decode_adsb_message(cleaned_message)
                if "Error" in result:
                    invalid_messages.append(result)
                else:
                    decoded_messages.append(result)
            else:
                invalid_messages.append({"Message": raw_message, "Error": "Invalid length"})
except FileNotFoundError:
    print(f"File not found: {log_file_path}")
    exit()

# Save decoded messages
with open(decoded_output_file, "w") as decoded_file:
    for msg in decoded_messages:
        decoded_file.write(str(msg) + "\n")

# Save invalid messages
with open(invalid_output_file, "w") as invalid_file:
    for msg in invalid_messages:
        invalid_file.write(str(msg) + "\n")

# Print summary
print(f"Decoded messages saved to {decoded_output_file}")
print(f"Invalid messages saved to {invalid_output_file}")

Decoded messages saved to decoded_messages.txt
Invalid messages saved to invalid_messages.txt
